# **Food Price analysis**

---

## Objectives

Create a single, analysis-ready dataset by:

* Loading and cleaning the two raw CSVs from the Global Food Price Inflation 2024 Kaggle archive

* Standardising column names, parsing dates, and coercing numeric fields

* Harmonising key categorical fields (e.g., country, item)

* Merging item-/series-level details with country-level context using reliable keys

* Saving cleaned per-file outputs and a merged dataset for downstream analysis

## Inputs

* data/WLD_RTFP_country_2023-10-02.csv

* data/WLD_RTP_details_2023-10-02.csv

* Python packages: pandas, numpy, os, re, pathlib

## Outputs

* data/WLD_RTFP_country_2023-10-02_clean.csv

* data/WLD_RTP_details_2023-10-02_clean.csv

* data/food_price_merged_cleaned.csv

* data/food_price_feature_engineered_clean.csv (final validated version)

## Additional Comments

* Join strategy: left join from details → country using keys:

1. ["country", "date", "item"]

2. ["country", "date"]

3. ["country"] (fallback)

* date comes from end_date_observations

# Change working directory

* We assume this notebook sits in a subfolder (e.g., jupyter_notebooks/). We make the parent the working directory.

In [1]:
# Access the current directory
import os
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/food-price-inflation-analysis/jupyter_notebooks'

In [2]:
# Make the parent directory current
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [3]:
# Confirm new current directory
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/food-price-inflation-analysis'

# Section 1

## Data Import

In [4]:
import pandas as pd

# Load dataset
country_path = "data/WLD_RTFP_country_2023-10-02.csv"
details_path = "data/WLD_RTP_details_2023-10-02.csv"

country_df = pd.read_csv(country_path)
details_df = pd.read_csv(details_path)

# Parse end_date_observations as datetime (add this line here)
details_df['end_date_observations'] = pd.to_datetime(details_df['end_date_observations'], format='%b %Y', errors='coerce')

---

# Section 2

## Defines reusable cleaning functions to:
- Standardise column names
- Tidy string values
- Parse date columns
- Convert numeric-looking strings to numbers
- Remove nearly empty columns
- Harmonise key categorical fields
- Provide quick summary reports

In [5]:
import re
import numpy as np
import pandas as pd

# Standardize column names: strip spaces, remove special characters, replace spaces with underscores, and lowercase
def normalise_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.replace(r"[^\w\s]", "", regex=True)
          .str.replace(r"\s+", "_", regex=True)
          .str.lower()
    )
    return df

# Clean string columns: strip leading/trailing spaces and collapse multiple spaces into one
def tidy_strings(df: pd.DataFrame, cols) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = (df[c].astype(str)
                           .str.strip()
                           .str.replace(r"\s+", " ", regex=True))
    return df

# Parse columns with names like 'date', 'month', 'year', or 'period' into datetime objects
def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Specify formats for known date columns
    date_formats = {
        "end_date_observations": "%b %Y",
        # Add more if needed
    }
    for c in df.columns:
        if re.search(r"(date|month|year|period)", c):
            fmt = date_formats.get(c)
            try:
                if fmt:
                    df[c] = pd.to_datetime(df[c], format=fmt, errors="coerce")
                else:
                    df[c] = pd.to_datetime(df[c], errors="coerce")
            except Exception:
                pass
    return df

# Convert columns that look numeric (even if stored as strings) to numeric dtype
def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if df[c].dtype == object:
            sample = df[c].dropna().astype(str).head(50)
            looks_numeric = (
                not sample.empty and
                sample.str.replace(",","", regex=False)
                      .str.replace("%","", regex=False)
                      .str.match(r"^-?\d+(\.\d+)?$")
                      .mean() > 0.6
            )
            if looks_numeric:
                df[c] = (df[c].astype(str)
                               .str.replace(",","", regex=False)
                               .str.replace("%","", regex=False)
                               .str.strip())
                df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Drop columns that are nearly empty (default: 98% or more missing values)
def drop_nearly_empty(df: pd.DataFrame, thresh=0.98) -> pd.DataFrame:
    na_ratio = df.isna().mean()
    to_drop = na_ratio[na_ratio >= thresh].index.tolist()
    return df.drop(columns=to_drop) if to_drop else df

# Run all cleaning steps in sequence for a generic DataFrame
def clean_generic(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out = normalise_columns(out)
    out = out.drop_duplicates()
    # Identify likely categorical columns and clean them
    cat_cols = [c for c in out.columns if out[c].dtype == object]
    priority = [c for c in ["country","country_name","item","commodity","product","unit","currency","series","market"] if c in out.columns]
    out = tidy_strings(out, list(dict.fromkeys(priority + cat_cols)))
    for c in ["country","country_name"]:
        if c in out.columns:
            out[c] = out[c].str.title()
    out = parse_dates(out)
    out = coerce_numeric(out)
    out = drop_nearly_empty(out, thresh=0.98)
    return out

# Print a brief summary report of the DataFrame's shape, duplicate count, and top missing columns
def brief_report(df: pd.DataFrame, name: str):
    print(f"{name}: shape={df.shape}, duplicates={df.duplicated().sum()}")
    miss = df.isna().sum()
    if miss.any():
        print("  top missing:", miss.sort_values(ascending=False).head(5).to_dict())

Defined reusable functions to standardise column names, clean string and numeric fields, parse dates, and drop nearly empty columns.

# Section 3

## Applying Data Cleaning Functions

Cleans both datasets using the utility functions, removes duplicates, standardises formats, and generates a brief report on shape, duplicates, and missing values for each file.

In [6]:
print(details_df.columns)

Index(['country', 'iso3', 'components', 'currency', 'start_date_observations',
       'end_date_observations', 'number_of_markets_modeled',
       'number_of_markets_covered', 'number_of_food_items',
       'number_of_observations_food', 'number_of_observations_other',
       'data_coverage_food', 'data_coverage_previous_12_months_food',
       'total_food_price_increase_since_start_date',
       'average_annualized_food_inflation', 'maximum_food_drawdown',
       'average_annualized_food_volatility',
       'average_monthly_food_price_correlation_between_markets',
       'average_annual_food_price_correlation_between_markets',
       'Rsquared_individual_food_items', 'Rsquared_individual_other_items',
       'index_confidence_score', 'imputation_model'],
      dtype='object')


In [7]:
print(details_df['end_date_observations'].head())

0   2023-09-01
1   2023-08-01
2   2023-07-01
3   2023-06-01
4   2023-08-01
Name: end_date_observations, dtype: datetime64[ns]


In [8]:
# Clean both datasets using the generic cleaning function
country_clean = clean_generic(country_df)
details_clean = clean_generic(details_df)

# Create unified date from end_date_observations
for df in [country_clean, details_clean]:
    if "end_date_observations" in df.columns:
        df["date"] = df["end_date_observations"]
    elif "start_date_observations" in df.columns:
        df["date"] = df["start_date_observations"]

# Print a summary report for each cleaned DataFrame
brief_report(country_clean, "Country (clean)")
brief_report(details_clean, "Details (clean)")

Country (clean): shape=(4798, 8), duplicates=0
  top missing: {'inflation': 364, 'open': 64, 'high': 64, 'low': 64, 'close': 64}
Details (clean): shape=(25, 22), duplicates=0


/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_56340/1962588202.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_56340/1962588202.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_56340/1962588202.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")


Applied the cleaning functions to both datasets, removed duplicates, and generated summary reports to check for missing values.

# Section 4 

## Saving Cleaned Data

After cleaning, we save the processed DataFrames as new CSV files in the `data/` directory.  
This ensures that the cleaned datasets are available for downstream analysis or sharing, without overwriting the original raw files.  
The file names include `_clean` to indicate they have been processed.

In [9]:
# Save cleaned versions alongside the raw files in 'data/'
country_out = "data/WLD_RTFP_country_2023-10-02_clean.csv"
details_out = "data/WLD_RTP_details_2023-10-02_clean.csv"

country_clean.to_csv(country_out, index=False)
details_clean.to_csv(details_out, index=False)

print("Saved:", country_out)
print("Saved:", details_out)

Saved: data/WLD_RTFP_country_2023-10-02_clean.csv
Saved: data/WLD_RTP_details_2023-10-02_clean.csv


# Section 5

# Conclusion & Summary

In this notebook, I have taken raw food price inflation data and transformed it into a clean, analysis-ready dataset through a series of structured ETL (Extract, Transform, Load) steps:

- **Data Cleaning:** Loaded the raw CSV files, standardized column names, parsed dates, coerced numeric fields, and harmonized key categorical variables.
- **Quality Assurance:** Checked for duplicates and missing values, ensuring the integrity of the cleaned data.
- **Merging:** Combined country-level and item-level datasets using robust join keys, resolving duplicate key issues to prevent data inflation.
- **Feature Engineering:** Created new variables such as time-based features, percentage changes, rolling averages, one-hot encodings, and missing value flags to enrich the dataset for analysis and modeling.
- **Final Validation:** Performed comprehensive data quality checks and cleaned up any problematic columns

This process ensures that the final dataset is tidy, consistent, and rich in features, making it suitable for exploratory data analysis, visualization, and predictive modeling. You can now confidently use `data/food_price_feature_engineered_clean.csv` for further insights and machine learning tasks.

In [10]:
# Prepare datasets for merging
def add_country_key(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "country" not in df.columns and "country_name" in df.columns:
        df["country"] = df["country_name"]
    return df

def ensure_date(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "date" not in df.columns:
        for alt in ["month","period","year"]:
            if alt in df.columns and pd.api.types.is_datetime64_any_dtype(df[alt]):
                df["date"] = df[alt]
                break
    return df

# Prepare datasets
country_keyed = ensure_date(add_country_key(country_clean))
details_keyed = ensure_date(add_country_key(details_clean))

# Select join keys automatically
candidate_key_sets = [["country","date","item"], ["country","date"], ["country"]]
chosen = None
for keys in candidate_key_sets:
    if all(k in country_keyed.columns for k in keys) and all(k in details_keyed.columns for k in keys):
        chosen = keys
        break

if not chosen:
    raise ValueError("No suitable join keys found.")

print("Merging on keys:", chosen)

# Check and resolve duplicate keys
country_dups = country_keyed.duplicated(subset=chosen).sum()
details_dups = details_keyed.duplicated(subset=chosen).sum()
print(f"Duplicate keys - Country: {country_dups}, Details: {details_dups}")

if country_dups > 0:
    country_keyed = country_keyed.drop_duplicates(subset=chosen)
    print(f"Resolved {country_dups} duplicate keys in country dataset")

# Perform merge
merged = pd.merge(
    details_keyed,
    country_keyed,
    on=chosen,
    how="left",
    suffixes=("", "_country")
).drop_duplicates()

print(f"Merged shape: {merged.shape}")
print(f"Missing matches: {merged['country'].isna().sum()}")

Merging on keys: ['country', 'date']
Duplicate keys - Country: 0, Details: 0
Merged shape: (25, 28)
Missing matches: 0


## Resolving Duplicate Keys Before Merging

When I check for duplicate keys before merging, I see that there are 4773 duplicate key rows in my `country_keyed` DataFrame and 0 in `details_keyed`. This means that, for my chosen join keys, the country-level data is not unique. If I merge as-is, each row in `details_keyed` will join with all matching rows in `country_keyed`, which could inflate my merged dataset and introduce errors.

Additionally, I see there are no missing values for the 'country' key in either DataFrame, so I don't need to worry about missing join keys.

Now that I've identified duplicate keys in my `country_keyed` DataFrame, I need to resolve them before merging. This helps prevent data inflation and ensures the merged dataset is accurate.

If the duplicate rows are exact matches, I can safely drop them. If they differ, I'll need to review and decide how to aggregate or select a representative row for each key.

# Section 6

## Save Merged Cleaned File & Quick Quality Assurance

Now that I've resolved duplicate keys and completed the merge, I will save the final cleaned dataset to disk.  
I'll also run a quick quality assurance check by summarizing numeric columns and showing the most frequent values for key categorical columns.  
This helps ensure the merged data is ready for downstream analysis and gives me a first look at the data distribution.

In [11]:
# Save the merged cleaned dataset
merged_out = "data/food_price_merged_cleaned.csv"
merged.to_csv(merged_out, index=False)
print("Saved merged cleaned dataset:", merged_out)

# Quick numeric summary (first 10 rows of summary only to keep output tidy)
import numpy as np
num_summary = merged.select_dtypes(include=[np.number]).describe().T.head(10)
display(num_summary)

# Frequent categories to guide EDA
for key in ["country", "item", "unit", "currency", "series"]:
    if key in merged.columns:
        print(f"\nTop values for `{key}`:")
        display(merged[key].value_counts().head(10))

Saved merged cleaned dataset: data/food_price_merged_cleaned.csv


,count,mean,std,min,25%,50%,75%,max
number_of_markets_modeled,25.0,56.4800,47.682212,9.00,24.00,42.00,79.00,228.00
number_of_markets_covered,25.0,56.4800,47.682212,9.00,24.00,42.00,79.00,228.00
number_of_food_items,25.0,9.3200,7.016172,3.00,4.00,7.00,11.00,26.00
data_coverage_food,25.0,35.3404,17.451470,8.84,21.55,31.08,47.97,69.91
average_annualized_food_inflation,25.0,12.1988,15.052045,1.24,3.58,6.68,10.79,55.30
maximum_food_drawdown,25.0,-22.8580,11.123922,-40.67,-31.98,-23.71,-13.96,-2.79
average_annualized_food_volatility,25.0,10.5904,5.413819,1.84,7.15,9.89,12.58,24.77
average_annual_food_price_correlation_between_markets,25.0,0.7756,0.186192,0.27,0.67,0.82,0.91,0.99
index_confidence_score,25.0,0.9196,0.040772,0.82,0.89,0.93,0.95,0.99
open,25.0,7.0976,16.762704,1.07,1.40,1.45,2.57,77.30



Top values for `country`:


country
Afghanistan             1
Liberia                 1
Chad                    1
Syrian Arab Republic    1
South Sudan             1
Somalia                 1
Sudan                   1
Nigeria                 1
Niger                   1
Mozambique              1
Name: count, dtype: int64


Top values for `currency`:


currency
XOF    4
XAF    4
AFN    1
MMK    1
SYP    1
SSP    1
SOS    1
SDG    1
NGN    1
MZN    1
Name: count, dtype: int64

# Section 7

## Feature Engineering

Create new variables to enhance the dataset for analysis and modeling, grouped into logical categories.

## 7.1 Extract year and month from a date column

If date exists, split into numeric year and month for seasonal/time-based analysis.

In [12]:
if "date" in merged.columns and pd.api.types.is_datetime64_any_dtype(merged["date"]):
    merged["year"] = merged["date"].dt.year
    merged["month"] = merged["date"].dt.month
    print("Added year and month columns from date.")

Added year and month columns from date.


## 7.2 Calculate month-on-month percentage change for close price

Tracks short-term price momentum per country.

In [13]:
if {"country", "date", "close"}.issubset(merged.columns):
    merged = merged.sort_values(["country", "date"])
    merged["close_pct_change"] = merged.groupby("country")["close"].pct_change()
    print("Added month-on-month close percentage change.")

Added month-on-month close percentage change.


## 7.3 Calculate a 3-month rolling average for close

Smooths price fluctuations for trend analysis.

In [14]:
if {"country", "close"}.issubset(merged.columns):
    merged["close_rolling_avg_3"] = (
        merged.groupby("country")["close"]
              .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
    )
    print("Added 3-month rolling average for close price.")

Added 3-month rolling average for close price.


## 7.4 Calculate year-on-year percentage change for close

Highlights long-term trends by comparing to the same month last year.

In [15]:
if {"country", "date", "close"}.issubset(merged.columns):
    merged["close_yoy_change"] = merged.groupby("country")["close"].pct_change(periods=12)
    print("Added year-on-year close percentage change.")

Added year-on-year close percentage change.


## 7.5 One-hot encode the item column

Converts item categories into separate binary columns for modelling.

In [16]:
if "item" in merged.columns:
    merged = pd.get_dummies(merged, columns=["item"], prefix="item")
    print("One-hot encoded the item column.")

## 7.6 Create inflation bands

Groups inflation values into named categories.

In [17]:
if "inflation" in merged.columns:
    bins = [-float("inf"), 0, 5, 10, float("inf")]
    labels = ["deflation", "low", "medium", "high"]
    merged["inflation_band"] = pd.cut(merged["inflation"], bins=bins, labels=labels)
    print("Added inflation bands.")

Added inflation bands.


## 7.7 Calculate average close price per country-item

Captures cross-category pricing patterns.

In [18]:
if {"country", "item", "close"}.issubset(merged.columns):
    merged["country_item_avg_close"] = (
        merged.groupby(["country", "item"])["close"].transform("mean")
    )
    print("Added average close price per country-item.")

## 7.8 Calculate days since last observation

Flags irregular time gaps in the data.

In [19]:
if {"country", "date"}.issubset(merged.columns) and pd.api.types.is_datetime64_any_dtype(merged["date"]):
    merged = merged.sort_values(["country", "date"])
    merged["days_since_last"] = merged.groupby("country")["date"].diff().dt.days
    print("Added days since last observation.")

Added days since last observation.


## 7.9 Flag missing inflation values

Allows models to consider “missingness” as a potential signal.

In [20]:
if "inflation" in merged.columns:
    merged["inflation_missing"] = merged["inflation"].isna().astype(int)
    print("Added missing value flag for inflation.")

Added missing value flag for inflation.


# Section 8

## Preparing the Feature Engineered Dataset

After completing my feature engineering process, I need to optimize the dataset for analysis. In this step, I apply column reduction to remove redundant, technical, and low-value columns that aren't needed for visualization and basic analysis.

My approach is to:
- Remove redundant date columns while keeping the main 'date' field
- Drop technical modeling columns that are only useful for advanced statistical analysis
- Remove correlation columns that add complexity without immediate value
- Eliminate failed feature engineering columns (like unsuccessful percentage changes)
- Drop one-hot encoded item columns with very low frequency (less than 10 occurrences)

This process reduces the dataset from 36 columns to approximately 18 columns, making it more manageable while retaining all the essential information for food price inflation analysis.

I will perform validation and cleanup before saving the final dataset in Section 10.


In [21]:
## Preparing the Feature Engineered Dataset

# Apply column reduction before validation
print(f"Dataset shape before column reduction: {merged.shape}")

# Define columns to drop
columns_to_drop = [
    # Redundant date columns (keep 'date' instead)
    'end_date_observations', 'start_date_observations',
    
    # Technical/modeling columns not needed for visualization
    'number_of_markets_modeled', 'number_of_markets_covered',
    'number_of_observations_food', 'number_of_observations_other',
    'rsquared_individual_food_items', 'rsquared_individual_other_items',
    'index_confidence_score', 'imputation_model',
    
    # Correlation columns
    'average_monthly_food_price_correlation_between_markets',
    'average_annual_food_price_correlation_between_markets',
    
    # Less useful engineered columns for basic visualization
    'close_yoy_change', 'country_item_avg_close', 'days_since_last'
]

# Drop one-hot encoded item columns with low frequency (less than 10 occurrences)
item_cols_to_drop = [col for col in merged.columns if col.startswith('item_') and merged[col].sum() < 10]
columns_to_drop.extend(item_cols_to_drop)

# Remove columns that actually exist in the dataset
columns_to_drop = [col for col in columns_to_drop if col in merged.columns]

print(f"Columns to drop: {columns_to_drop}")

# Drop the columns and duplicates
merged_reduced = merged.drop(columns=columns_to_drop)
merged_reduced = merged_reduced.drop_duplicates()

print(f"Dataset shape after column reduction: {merged_reduced.shape}")
print(f"Dropped {len(columns_to_drop)} columns")

# Proceeding to validation and final cleanup in Section 10
print("Dataset prepared for validation and final save...")

Dataset shape before column reduction: (25, 36)
Columns to drop: ['end_date_observations', 'start_date_observations', 'number_of_markets_modeled', 'number_of_markets_covered', 'number_of_observations_food', 'number_of_observations_other', 'rsquared_individual_food_items', 'rsquared_individual_other_items', 'index_confidence_score', 'imputation_model', 'average_monthly_food_price_correlation_between_markets', 'average_annual_food_price_correlation_between_markets', 'close_yoy_change', 'days_since_last']
Dataset shape after column reduction: (25, 22)
Dropped 14 columns
Dataset prepared for validation and final save...


# Section 9

## Final Dataset Validation

Before considering my ETL process complete, I perform a comprehensive validation check to ensure data quality. This validation process examines multiple aspects of the dataset:

### Data Quality Checks I Perform:

1. **Duplicate Column Names**: I check if any columns have identical names, which would cause issues in analysis
2. **Identical Content**: I identify columns that contain exactly the same data (redundant columns)
3. **Empty Columns**: I find columns that are completely null or empty
4. **High Missing Values**: I flag columns with more than 95% missing data
5. **Single-Value Columns**: I identify columns with no variance (only one unique value)
6. **Suspicious Patterns**: I look for unnamed columns or index-like artifacts

### Dataset Summary:

The validation also provides a comprehensive summary including:
- Dataset shape and memory usage
- Column type distribution
- Complete list of all columns for manual review

This step is crucial because it reveals any remaining data quality issues that could affect downstream analysis.

In [22]:
# Final Dataset Validation Check
def final_dataset_check(df, dataset_name="Dataset"):
    """
    Comprehensive final check for duplicate columns, irrelevant columns, and data quality
    """
    print(f"=== FINAL VALIDATION CHECK FOR {dataset_name.upper()} ===\n")
    
    # 1. Check for duplicate column names
    duplicate_cols = df.columns[df.columns.duplicated()].tolist()
    if duplicate_cols:
        print(f"⚠️ DUPLICATE COLUMN NAMES FOUND: {duplicate_cols}")
    else:
        print("✅ No duplicate column names")
    
    # 2. Check for columns with identical content
    identical_cols = []
    cols = df.columns.tolist()
    for i in range(len(cols)):
        for j in range(i+1, len(cols)):
            if df[cols[i]].equals(df[cols[j]]):
                identical_cols.append((cols[i], cols[j]))
    
    if identical_cols:
        print(f"⚠️ COLUMNS WITH IDENTICAL CONTENT: {identical_cols}")
    else:
        print("✅ No columns with identical content")
    
    # 3. Check for columns that are all null/empty
    empty_cols = df.columns[df.isnull().all()].tolist()
    if empty_cols:
        print(f"⚠️ COMPLETELY EMPTY COLUMNS: {empty_cols}")
    else:
        print("✅ No completely empty columns")
    
    # 4. Check for columns with very high missing values (>95%)
    high_missing = df.columns[df.isnull().mean() > 0.95].tolist()
    if high_missing:
        print(f"⚠️ COLUMNS WITH >95% MISSING VALUES: {high_missing}")
    else:
        print("✅ No columns with excessive missing values")
    
    # 5. Check for single-value columns (no variance)
    single_value_cols = []
    for col in df.columns:
        if df[col].nunique() <= 1:
            single_value_cols.append(col)
    
    if single_value_cols:
        print(f"⚠️ SINGLE-VALUE COLUMNS (NO VARIANCE): {single_value_cols}")
    else:
        print("✅ No single-value columns")
    
    # 6. Check for suspicious column patterns
    suspicious_cols = []
    for col in df.columns:
        # Check for unnamed columns
        if 'unnamed' in col.lower():
            suspicious_cols.append(f"{col} (unnamed)")
        # Check for index-like columns
        if col.lower() in ['index', 'level_0', 'level_1']:
            suspicious_cols.append(f"{col} (index-like)")
    
    if suspicious_cols:
        print(f"⚠️ SUSPICIOUS COLUMNS: {suspicious_cols}")
    else:
        print("✅ No suspicious column patterns")
    
    # 7. Summary statistics
    print(f"\n📊 DATASET SUMMARY:")
    print(f"   Shape: {df.shape}")
    print(f"   Total columns: {len(df.columns)}")
    print(f"   Total rows: {len(df)}")
    print(f"   Duplicate rows: {df.duplicated().sum()}")
    print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # 8. Column types summary
    print(f"\n📋 COLUMN TYPES:")
    print(df.dtypes.value_counts().to_dict())
    
    # 9. Show all column names for manual review
    print(f"\n📝 ALL COLUMNS ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i:2d}. {col}")
    
    return df

# Run the final check on your feature engineered dataset
final_dataset_check(merged_reduced, "Food Price Feature Engineered")

=== FINAL VALIDATION CHECK FOR FOOD PRICE FEATURE ENGINEERED ===

✅ No duplicate column names
⚠️ COLUMNS WITH IDENTICAL CONTENT: [('iso3', 'iso3_country'), ('close', 'close_rolling_avg_3')]
⚠️ COMPLETELY EMPTY COLUMNS: ['close_pct_change']
⚠️ COLUMNS WITH >95% MISSING VALUES: ['close_pct_change']
⚠️ SINGLE-VALUE COLUMNS (NO VARIANCE): ['close_pct_change', 'inflation_missing']
✅ No suspicious column patterns

📊 DATASET SUMMARY:
   Shape: (25, 22)
   Total columns: 22
   Total rows: 25
   Duplicate rows: 0
   Memory usage: 0.02 MB

📋 COLUMN TYPES:
{dtype('float64'): 11, dtype('O'): 5, dtype('int64'): 2, dtype('int32'): 2, dtype('<M8[ns]'): 1, CategoricalDtype(categories=['deflation', 'low', 'medium', 'high'], ordered=True, categories_dtype=object): 1}

📝 ALL COLUMNS (22):
    1. country
    2. iso3
    3. components
    4. currency
    5. number_of_food_items
    6. data_coverage_food
    7. average_annualized_food_inflation
    8. maximum_food_drawdown
    9. average_annualized_food_vol

,country,iso3,components,currency,number_of_food_items,data_coverage_food,average_annualized_food_inflation,maximum_food_drawdown,average_annualized_food_volatility,date,...,low,close,inflation,iso3_country,year,month,close_pct_change,close_rolling_avg_3,inflation_band,inflation_missing
0,Afghanistan,AFG,"Bread (1 KG, Index Weight = 1), Rice (Low Qual...",AFN,3,31.77,6.06,-40.67,7.93,2023-09-01,...,1.42,1.42,-6.50,AFG,2023,9,NaN,1.42,deflation,0
2,Burkina Faso,BFA,"Maize (White) (1 KG, Index Weight = 1), Millet...",XOF,3,55.20,6.81,-36.70,13.71,2023-07-01,...,1.42,1.45,-15.91,BFA,2023,7,NaN,1.45,deflation,0
1,Burundi,BDI,"Bananas (1 KG, Index Weight = 1), Beans (1 KG,...",BIF,10,38.24,7.86,-30.77,12.03,2023-08-01,...,1.87,1.88,26.79,BDI,2023,8,NaN,1.88,high,0
4,Cameroon,CMR,"Bananas (12 KG, Index Weight = 0.08), Cassava ...",XAF,10,8.84,2.47,-2.79,1.84,2023-08-01,...,1.28,1.29,2.28,CMR,2023,8,NaN,1.29,low,0
3,Central African Republic,CAF,"Cassava (Cossette) (1 KG, Index Weight = 1), M...",XAF,5,24.85,5.22,-24.85,13.74,2023-06-01,...,1.36,1.43,0.04,CAF,2023,6,NaN,1.43,low,0
23,Chad,TCD,"Maize (White) (1 KG, Index Weight = 1), Millet...",XAF,4,27.41,3.16,-32.67,12.58,2023-08-01,...,1.38,1.40,0.99,TCD,2023,8,NaN,1.40,low,0
5,"Congo, Dem. Rep.",COD,"Beans (1 KG, Index Weight = 1), Cassava (Cosse...",CDF,17,15.65,6.64,-15.73,6.99,2023-01-01,...,1.39,1.39,14.09,COD,2023,1,NaN,1.39,high,0
6,"Congo, Rep.",COG,"Beans (White) (1 KG, Index Weight = 1), Cassav...",XAF,4,29.37,1.24,-8.13,5.45,2022-04-01,...,1.05,1.06,4.14,COG,2022,4,NaN,1.06,low,0
7,"Gambia, The",GMB,"Apples (Red) (1 KG, Index Weight = 1), Bananas...",GMD,26,31.08,6.68,-20.83,7.15,2023-07-01,...,1.69,1.70,21.56,GMB,2023,7,NaN,1.70,high,0
8,Guinea-Bissau,GNB,"Millet (1 KG, Index Weight = 1), Oil (Vegetabl...",XOF,5,15.61,1.62,-28.04,11.77,2023-08-01,...,1.42,1.43,3.81,GNB,2023,8,NaN,1.43,low,0


# Section 10

## Final Data Quality Cleanup and Save

When my validation check identifies problematic columns, I perform automatic cleanup to resolve these issues before saving the final dataset:

### Issues I Address:

1. **Duplicate Content Columns**: I remove `iso3_country` (duplicate of `iso3`) and `close_rolling_avg_3` (identical to `close`)
2. **Empty Columns**: I remove `close_pct_change` which contains no data
3. **Single-Value Columns**: I remove `inflation_missing` if it has no variance

### My Cleanup Process:

- I create a list of problematic columns to remove
- I check if each column actually exists before attempting to remove it
- I drop the problematic columns and create a cleaned dataset
- I re-run the validation check to confirm all issues are resolved
- I save the final validated dataset

This ensures that my final dataset is completely clean and ready for analysis, with no duplicate or problematic columns that could cause issues in visualization or modeling.

### Final Output:

The result is a pristine dataset saved as `data/food_price_feature_engineered_clean.csv` that passes all data quality checks and is optimized for food price inflation analysis and visualization. This is the final, analysis-ready dataset that contains all validated features.

In [23]:
# Clean up problematic columns in the feature engineered dataset

# 1. Remove duplicate columns
columns_to_remove = [
    'iso3_country',  # duplicate of iso3
    'close_pct_change',  # completely empty
    'close_rolling_avg_3'  # identical to close
]

# 2. Check if inflation_missing has any variance before removing
if merged_reduced['inflation_missing'].nunique() <= 1:
    columns_to_remove.append('inflation_missing')

# 3. Remove the problematic columns
merged_cleaned = merged_reduced.drop(columns=[col for col in columns_to_remove if col in merged_reduced.columns])

print(f"Removed {len([col for col in columns_to_remove if col in merged_reduced.columns])} problematic columns")
print(f"New dataset shape: {merged_cleaned.shape}")

# 4. Re-run the validation check
final_dataset_check(merged_cleaned, "Food Price Feature Engineered - Cleaned")

# 5. Save the truly cleaned dataset
cleaned_path = "data/food_price_feature_engineered_clean.csv"
merged_cleaned.to_csv(cleaned_path, index=False)
print(f"Cleaned dataset saved to: {cleaned_path}")

Removed 4 problematic columns
New dataset shape: (25, 18)
=== FINAL VALIDATION CHECK FOR FOOD PRICE FEATURE ENGINEERED - CLEANED ===

✅ No duplicate column names
✅ No columns with identical content
✅ No completely empty columns
✅ No columns with excessive missing values
✅ No single-value columns
✅ No suspicious column patterns

📊 DATASET SUMMARY:
   Shape: (25, 18)
   Total columns: 18
   Total rows: 25
   Duplicate rows: 0
   Memory usage: 0.02 MB

📋 COLUMN TYPES:
{dtype('float64'): 9, dtype('O'): 4, dtype('int32'): 2, dtype('int64'): 1, dtype('<M8[ns]'): 1, CategoricalDtype(categories=['deflation', 'low', 'medium', 'high'], ordered=True, categories_dtype=object): 1}

📝 ALL COLUMNS (18):
    1. country
    2. iso3
    3. components
    4. currency
    5. number_of_food_items
    6. data_coverage_food
    7. average_annualized_food_inflation
    8. maximum_food_drawdown
    9. average_annualized_food_volatility
   10. date
   11. open
   12. high
   13. low
   14. close
   15. inflatio

# Conclusion & Summary

In this notebook, I have taken raw food price inflation data and transformed it into a clean, analysis-ready dataset through a series of structured ETL (Extract, Transform, Load) steps:

- **Data Cleaning:** Loaded the raw CSV files, standardized column names, parsed dates, coerced numeric fields, and harmonized key categorical variables.
- **Quality Assurance:** Checked for duplicates and missing values, ensuring the integrity of the cleaned data.
- **Merging:** Combined country-level and item-level datasets using robust join keys, resolving duplicate key issues to prevent data inflation.
- **Feature Engineering:** Created new variables such as time-based features, percentage changes, rolling averages, one-hot encodings, and missing value flags to enrich the dataset for analysis and modeling.
- **Final Validation:** Performed comprehensive data quality checks and cleaned up any problematic columns

This process ensures that the final dataset is tidy, consistent, and rich in features, making it suitable for exploratory data analysis, visualization, and predictive modeling. You can now confidently use `data/food_price_feature_engineered_clean.csv` for further insights and machine learning tasks.

---